In [1]:
import torch
import nibabel as nib
import os
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
import numpy as np
import time
from tqdm.auto import tqdm
import utils as utils
from scipy.ndimage import rotate

/home/student/sinaziaee/mini_conda/miniconda3/envs/test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_current_path(path=None):
    if path:
        return path
    else:
        return os.getcwd()

new_folder = 'data_npy2'

if not os.path.exists(f'{get_current_path()}/{new_folder}'):
    os.mkdir(f'{get_current_path()}/{new_folder}')
    
if not os.path.exists(f'{get_current_path()}/{new_folder}/train'):
    os.mkdir(f'{get_current_path()}/{new_folder}/train')
    
if not os.path.exists(f'{get_current_path()}/{new_folder}/valid'):
    os.mkdir(f'{get_current_path()}/{new_folder}/valid')
    
if not os.path.exists(f'{get_current_path()}/{new_folder}/test'):
    os.mkdir(f'{get_current_path()}/{new_folder}/test')
    
def make_path(path):
    if not os.path.exists(path):
        os.makedirs(path)
    return path

train_image_path = make_path(f'{get_current_path()}/{new_folder}/train/image')
train_segment_path = make_path(f'{get_current_path()}/{new_folder}/train/segmentation')
valid_image_path = make_path(f'{get_current_path()}/{new_folder}/valid/image')
valid_segment_path = make_path(f'{get_current_path()}/{new_folder}/valid/segmentation')
test_image_path = make_path(f'{get_current_path()}/{new_folder}/test/image')
test_segment_path = make_path(f'{get_current_path()}/{new_folder}/test/segmentation')

src_train_image_path = '3d_original_padded_images/training/images'
src_valid_image_path = '3d_original_padded_images/validation/images'
src_test_image_path = '3d_original_padded_images/testing/images' 

src_train_mask_path = '3d_original_padded_images/training/labels'
src_valid_mask_path = '3d_original_padded_images/validation/labels'
src_test_mask_path = '3d_original_padded_images/testing/labels' 

In [3]:
def perform_image_slicing(root_path, saving_path):
    count = 0
    depth_list = []
    for file_name in tqdm(sorted(os.listdir(root_path))):
        # print(os.path.join(root_path, file_name))
        if file_name.endswith('.nii.gz'):
            img_id = str(file_name.split('_')[1])
            img = nib.load(os.path.join(root_path, file_name)).get_fdata()
            # normalizing the image between 0 and 1
            img=(img-img.min())/(max((img.max()-img.min()),1e-3))
            depth = img.shape[2]
            depth_list.append(depth)
            for j in range(depth):
                new_path=os.path.join(saving_path, '{:05d}.npy'.format(j+count))
                new_img = torch.tensor(img[:, :, j:j+1].astype(np.float32))
                new_img = new_img.permute(2, 0, 1)
                new_img = np.array(new_img)
                new_img = np.squeeze(new_img, axis=0)
                new_img = rotate(new_img, 90)
                new_img = np.expand_dims(new_img, axis=0)
                np.save(new_path, new_img)
            count += depth
    return depth_list

train_depth_list = perform_image_slicing(root_path=src_train_image_path, saving_path=train_image_path)
valid_depth_list = perform_image_slicing(root_path=src_valid_image_path, saving_path=valid_image_path)
test_depth_list = perform_image_slicing(root_path=src_test_image_path, saving_path=test_image_path)

 50%|█████     | 5/10 [01:11<01:11, 14.23s/it]

In [ ]:
import json

depth_lists = {
    'train': train_depth_list,
    'valid': valid_depth_list,
    'test': test_depth_list
}

json_file_path = f'{new_folder}/depth_lists.json'

with open(json_file_path, 'w') as json_file:
    json.dump(depth_lists, json_file)


In [ ]:
import json

json_file_path = f'{new_folder}/depth_lists.json'

with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

print(data['train'])
print(min(data['train']), max(data['train']))
print(min(data['valid']), max(data['valid']))
print(min(data['test']), max(data['test']))

[240, 195, 216, 221, 223, 201, 205, 196, 206, 221, 214, 186, 231, 310, 223, 210, 218, 223, 310, 310, 310, 205, 310, 233, 310, 205, 227, 223, 198, 201, 251, 237, 310, 216, 198, 201, 206, 222, 310, 209, 218, 193, 215, 228, 310, 203, 223, 211, 213, 216, 209, 194, 310, 201, 224, 210, 233, 310, 209]
186 310
185 466
181 310


In [ ]:
import matplotlib.pyplot as plt
def perform_segment_slicing(root_path, saving_path):
    count = 0
    depth_list = []
    for file_name in tqdm(sorted(os.listdir(root_path))):
        # print(os.path.join(root_path, file_name))
        seg_id = str(file_name.split('_')[1]).split('.')[0]
        seg = nib.load(os.path.join(root_path, file_name)).get_fdata()
        seg_no_cancer=np.where(seg>0,1,0).astype(np.uint8)
        depth = seg_no_cancer.shape[2]
        depth_list.append(depth)
        for j in range(depth):
            new_path=os.path.join(saving_path, '{:05d}.npy'.format(j+count))
            seg_1ch=torch.tensor(seg_no_cancer[:, :,j:j+1],dtype=torch.int64)
            seg_1ch = np.array(seg_1ch)
            seg_1ch = np.squeeze(seg_1ch, axis=2)
            seg_1ch = rotate(seg_1ch, 90)
            seg_1ch = np.expand_dims(seg_1ch, axis=2)
            seg_1ch = torch.tensor(seg_1ch)
            seg_2ch=F.one_hot(seg_1ch,num_classes=2)
            seg_2ch=torch.squeeze(seg_2ch.permute(3,0,1,2))
            seg_2ch=np.array(seg_2ch,dtype=np.uint8)
            np.save(new_path,seg_2ch)
        count += depth

perform_segment_slicing(root_path=src_train_mask_path, saving_path=train_segment_path)
perform_segment_slicing(root_path=src_valid_mask_path, saving_path=valid_segment_path)
perform_segment_slicing(root_path=src_test_mask_path, saving_path=test_segment_path)

 63%|██████▎   | 37/59 [04:34<02:43,  7.42s/it]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

def plot_slices(imaging_path, segmentation_path):
    imaging_slice = np.load(imaging_path)
    segmentation_slice = np.load(segmentation_path)

    # Squeeze the singleton dimension if it exists
    imaging_slice = np.transpose(imaging_slice, (1, 2, 0))
    imaging_slice = np.squeeze(imaging_slice)

    fig, ax = plt.subplots()
    ax.imshow(imaging_slice, cmap='gray')
    ax.imshow(segmentation_slice[1], cmap='Reds', alpha=0.3)
    ax.set_title('Imaging Slice with Segmentation')
    ax.axis('off')
    plt.show()
    
    plt.imshow(imaging_slice, cmap='gray')
    plt.show()